# Prepare Base Model 

In [1]:
import os 
%pwd

'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPs Project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Data Science\\Codes\\02 MLOPs\\00 MLOPs Project'

In [3]:
# Create entity 
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path 
    updated_base_model_path : Path 
    params_image_size : list 
    params_learning_rate : float 
    params_include_top : bool 
    params_weight : str 
    params_classes : int 

In [4]:
from cnnClassifier.constants import CONFIG_PATH_YAML,PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml,create_directories
from cnnClassifier import logger 

In [5]:

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_PATH_YAML,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_roots])

    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model 
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weight=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [6]:
import os 
import urllib.request as request 
from zipfile import ZipFile 
import tensorflow as tf 

In [ ]:
class PrepareBaseModel:
    """Download VGG16 model from website Keras application"""
    def __init__(self,config : PrepareBaseModelConfig):
        """
        Config will take all information about model from config/config.yaml and params.yaml
        """
        self.config = config 
    
    def get_base_model(self):
        """This main method will download model from website"""
        self.model = tf.keras.applications.vgg16.VGG16(
            # shape of images according to which model will process
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight,
            # Top Fully connected layer of model(head) included or not
            include_top=self.config.params_include_top
        )

        # Save model at directory --------> base_model_path (config/config.yaml)
        self.save_model(path=self.config.base_model_path,model=self.model)